

---


# Instalando os Módulos, Importando Credenciais e Fazendo a Autenticação e Inicialização.

In [ ]:
#Instalar os módutos
!pip install ibm_watson wget
!pip install ffmpeg-python

In [2]:
#Realizando a autenticação e inicialização da API STT e TTS
from ibm_watson import SpeechToTextV1, TextToSpeechV1, AssistantV2
import json
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

from pandas import json_normalize 

from IPython.display import HTML, Audio
from google.colab.output import eval_js 
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

from scipy.io.wavfile import write #Biblioteca para salvar o áudio como arquivo

import IPython #para executar um áudio

In [3]:
#Importar credenciais Text to Speech
authenticatorTTS = IAMAuthenticator('v8PVX7rWrabxrssHdSgytiL-6QLTbK7z2PT2kzm_FAGg')
tts = TextToSpeechV1 ( authenticator=authenticatorTTS )
tts.set_service_url('https://api.us-south.text-to-speech.watson.cloud.ibm.com/instances/2b7eeaaa-5efc-4081-bad1-8bba8626a5eb')

#Importar credenciais Speech to Text
url_s2t = 'https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/050a3154-b768-43ef-a576-6cf26c725dc4'
iam_apikey_s2t = 'vvVSgkymm0LjqVJPLPis5x8gudY1eoUmKl_F8YlQzTed'
authenticator = IAMAuthenticator(iam_apikey_s2t)
s2t = SpeechToTextV1(authenticator=authenticator)
s2t.set_service_url(url_s2t)
s2t

# Set up Assistant service - IBM Watson Assistant.
authenticator = IAMAuthenticator('rA67lVHVIhqZrCb9YWemq2FT-S2H5f2Wr0UXfRn37Xul') #  API key
service = AssistantV2(
    version = '2019-02-28',
    authenticator = authenticator
)

assistant_id = '14f07f3a-81b5-4557-8f7e-c8e2686fa16a' # ID do assistente

# Realizando a autenticação e inicialização da api STT
authenticatorSTT = IAMAuthenticator(iam_apikey_s2t)
speech_to_text = SpeechToTextV1(authenticator=authenticatorSTT)
speech_to_text.set_service_url(url_s2t)
speech_to_text



---


# Utilizando a Biblioteca get_audio Para Gravar um Áudio no Colab

In [4]:
#Script e biblioteca get_audio para gravar um áudio direto pelo calab
AUDIO_HTML = """
<script>
var my_div = document.createElement('DIV');
var my_p = document.createElement('P');
var my_btn = document.createElement('BUTTON');
var t = document.createTextNode('Press to start recording');

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr



---


# Fazendo a criação do áudio que será reproduzido durante a convera.

In [27]:
#Crinado o audio para ser repoduzido na conversação
authenticatorTTS = IAMAuthenticator('v8PVX7rWrabxrssHdSgytiL-6QLTbK7z2PT2kzm_FAGg')
text_to_speech = TextToSpeechV1 ( authenticator=authenticatorTTS )

text_to_speech.set_service_url('https://api.us-south.text-to-speech.watson.cloud.ibm.com/instances/2b7eeaaa-5efc-4081-bad1-8bba8626a5eb')

with open('audio_praticar.wav', 'wb') as audio_file:
    audio_file.write(
        text_to_speech.synthesize(
            'how are you doing',
            voice='en-US_LisaV3Voice',
            accept='audio/wav'        
        ).get_result().content)




---


# Executando o progrma para iniciar a conversa com o usuário.

In [ ]:
# Criar sessão.
session_id = service.create_session( assistant_id = assistant_id ).get_result()['session_id']

# Começa uma conversa com mensagem vazia.
message_input = {
    'message_type:': 'text',
    'text': ''
    }

print('\n\nOrientações para a execução: \n - Escreva "ola" para ser educado(a) e inicar a prática. ᕙ(`▿´)ᕗ \n - Para finalizar a conversação antes do fim digite "sair", mas ficarei triste (╥﹏╥)\n\n')

frase_usuario = 'how are you doing'

# input/output loop
while message_input['text'] != 'sair':
    
    # Manda a mensagem pro assistente
    response = service.message( assistant_id, session_id, input = message_input ).get_result()

    # Imprima a saída do diálogo, se houver
    if response['output']['generic']:
        if response['output']['generic'][0]['response_type'] == 'text':
            print(response['output']['generic'][0]['text'])
    
    if response['output']['intents']:
        if response['output']['intents'][0]['intent'] == 'ola':
          IPython.display.Audio('audio_praticar.wav')

          #Recebendo o áudio
          audio, sr = get_audio()

          #Salvar nosso aúdio como um arquivo
          write('resposta.wav', sr, audio)

          #Passando o áudio para a API STT da IBM
          with open('resposta.wav', mode='rb')  as wav:
            response = s2t.recognize(audio=wav, content_type='audio/wav', model='en-US_BroadbandModel')

          #Fazendo a conversão do json e colocando em uma variável e retirando os espaçamentos
          json_normalize(response.result['results'],'alternatives')
          recognized_text=response.result['results'][0]['alternatives'][0]['transcript']
          type(recognized_text)
          found_words=recognized_text.split(' ')
          initial_words=frase_usuario.split(' ')
          wrong_words=set(initial_words) - set(found_words)

          #Fazendo a verificação dos erros
          total_palavras = len(initial_words)
          usuario_erros = len(wrong_words)
          usuario_acertos = (total_palavras - usuario_erros)
          print(recognized_text)

          print('Você acertou {} das {} palavras ditas'.format(usuario_acertos, total_palavras))

          if usuario_acertos != total_palavras:
            print('Não foi dessa vez, você errou a(s) palavra(s) {} tente novamente ( ͡❛  ͟ʖ ͡❛)'.format(wrong_words))
            break
            
          else:
            print('Parabéns vc falou corretamente (👍≖‿‿≖)👍')
            break

    # Entrada para nova mensagem do usuário
    user_input = input('>> ')
    message_input = {'text': user_input}


# Saindo do loop principal, concluímos a conversa
service.delete_session( assistant_id = assistant_id, session_id = session_id )